# Definir os caminhos (padrão novo)

In [0]:
val BASE   = "abfss://imoveis@datalakealura.dfs.core.windows.net"
val INBOUND = s"$BASE/inbound"
val BRONZE  = s"$BASE/bronze"

val arquivoInbound = s"$INBOUND/dados_brutos_imoveis.json"



# Validar que o arquivo existe no inbound

In [0]:
dbutils.fs.ls(INBOUND)



# Ler o JSON

In [0]:
val dados = spark.read.json(arquivoInbound)
display(dados)



# Remover colunas irrelevantes (imagens, usuario)

In [0]:
val dadosAnuncio = dados.drop("imagens", "usuario")
display(dadosAnuncio)


# Criar a coluna id a partir de anuncio.id

In [0]:
import org.apache.spark.sql.functions.col

val dfBronze = dadosAnuncio.withColumn("id", col("anuncio.id").cast("string"))
display(dfBronze)



# Salvar em Delta na camada Bronze (overwrite)

In [0]:
import org.apache.spark.sql.SaveMode

val pathBronze = s"$BRONZE/dataset_imoveis"

dfBronze.write
  .format("delta")
  .mode(SaveMode.Overwrite)
  .save(pathBronze)


# Validar gravação (listar e ler de volta)

In [0]:
dbutils.fs.ls(pathBronze)


In [0]:
val bronzeLido = spark.read.format("delta").load(pathBronze)
display(bronzeLido)

